# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import keras
from keras import layers
import keras_tuner as kt
from keras import backend as K
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import csv

# Dataset

In [ ]:
df = pd.read_csv("C:/Users/Rolando/Desktop/FREDMDApril19.csv")
df.head()

# Split the data

In [ ]:
n = len(df)
train=df.iloc[:int(0.7*(n-5)),]
val=df.iloc[int(0.7*(n-5)):(n-5),]
test=df.iloc[(n-5):n,]
Varnames=df.columns
Housing=['HOUST','HOUSTNE','HOUSTMW','HOUSTS','HOUSTW','PERMIT','PERMITNE','PERMITMW','PERMITS','PERMITW']

In [ ]:
test

# Evolution of Housing features over time

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df[Housing],label=Housing)
plt.xticks(ticks=[118+120*i for i in range(5)], labels=[1970+10*i for i in range(5)])
plt.axvline(x=len(train), color='black',linestyle='dashed', alpha=0.7)
plt.axvline(x=n-5, color='black',linestyle='dashed', alpha=0.7)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

# ACFs and PACFs

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(15, 20))
for i in range(10):
    plot_acf(train[Housing].iloc[:,i], ax=axes[i%5,0+2*(i%2)])
    axes[i%5,0+2*(i%2)].set_title(f'ACF for {Housing[i]}')
    plot_pacf(train[Housing].iloc[:,i], ax=axes[i%5,1+2*(i%2)])
    axes[i%5,1+2*(i%2)].set_title(f'PACF for {Housing[i]}')
plt.tight_layout()
plt.show()

# Normalize data

In [ ]:
train_mean = train.mean()
train_std = train.std() 

train_df = (train - train_mean) / train_std
val_df = (val - train_mean) / train_std
test_df = (test - train_mean) / train_std

# Window

In [ ]:
def WindowData(lag, train_df=train_df,val_df=val_df,test_df=test_df,val=True):
    time_steps_lag=lag
    time_steps_h=5

    # Reshape train

    n_samples = train_df.shape[0] - time_steps_lag - time_steps_h + 1
    n_features = train_df.shape[1]
    n_features_y = 10

    X = np.zeros((n_samples, time_steps_lag, n_features))
    y = np.zeros((n_samples, time_steps_h, n_features_y))

    for i in range(n_samples):
        X[i] = train_df[i:i+time_steps_lag]
        y[i] = train_df[Housing][i+time_steps_lag:i+time_steps_lag+time_steps_h]

    train_X = X
    train_Y = y

    if val:
        # Reshape val

        n_samples = val_df.shape[0] - time_steps_h + 1
        n_features = val_df.shape[1]
        n_features_y = 10

        X = np.zeros((n_samples, time_steps_lag, n_features))
        y = np.zeros((n_samples, time_steps_h, n_features_y))

        for i in range(n_samples):
            if (i==0):
                X[i]=train_df[train_df.shape[0]-time_steps_lag:train_df.shape[0]]
            elif (i<time_steps_lag):
                X[i,0:time_steps_lag-i] = train_df[train_df.shape[0]-time_steps_lag+i:train_df.shape[0]]
                X[i,time_steps_lag-i:time_steps_lag] = val_df[time_steps_lag-i:time_steps_lag]
            else:
                X[i] = val_df[i-time_steps_lag:i]
            y[i] = val_df[Housing][i:i+time_steps_h]

        val_X = X
        val_Y = y

        # Reshape test

        n_samples = test.shape[0] - time_steps_h + 1
        n_features = test.shape[1]
        n_features_y = 10

        X = np.zeros((n_samples, time_steps_lag, n_features))
        y = np.zeros((n_samples, time_steps_h, n_features_y))

        for i in range(n_samples):
            if (i==0):
                X[i]=val_df[val_df.shape[0]-time_steps_lag:val_df.shape[0]]
            elif (i<time_steps_lag):
                X[i,0:time_steps_lag-i] = val_df[val_df.shape[0]-time_steps_lag+i:val_df.shape[0]]
                X[i,time_steps_lag-i:time_steps_lag] = test_df[time_steps_lag-i:time_steps_lag]
            else:
                X[i] = test_df[i-time_steps_lag:i]
            y[i] = test_df[Housing][i:i+time_steps_h]

        test_X = X
        test_Y = y
        return train_X, train_Y, val_X, val_Y, test_X, test_Y
    else :
                # Reshape test

        n_samples = test.shape[0] - time_steps_h + 1
        n_features = test.shape[1]
        n_features_y = 10

        X = np.zeros((n_samples, time_steps_lag, n_features))
        y = np.zeros((n_samples, time_steps_h, n_features_y))

        for i in range(n_samples):
            if (i==0):
                X[i]=train_df[train_df.shape[0]-time_steps_lag:train_df.shape[0]]
            elif (i<time_steps_lag):
                X[i,0:time_steps_lag-i] = train_df[train_df.shape[0]-time_steps_lag+i:train_df.shape[0]]
                X[i,time_steps_lag-i:time_steps_lag] = test_df[time_steps_lag-i:time_steps_lag]
            else:
                X[i] = test_df[i-time_steps_lag:i]
            y[i] = test_df[Housing][i:i+time_steps_h]

        test_X = X
        test_Y = y
        return train_X, train_Y, test_X, test_Y
        

# LSTM

In [ ]:
def model_builder(hp,seedp=0):
    # Clear session
    K.clear_session()
    # Seeds
    keras.utils.set_random_seed(100510766+seedp)
    # Model
    model = tf.keras.models.Sequential()
    # Hyperparameters
    hp_units = hp.Choice('units', values=[32,64,128,256,512,1024])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 
    hp_activation = hp.Choice('activation', values=['relu','tanh']) 
    hp_dropout=hp.Choice('dropout',values=[float(0), 0.1, 0.2, 0.3]) 
    hp_kernel_reg = hp.Choice('kernel_regularizer', values=[float(0), 1e-3, 1e-2, 1e-1])
    # Layers
    model.add(tf.keras.layers.LSTM(units=hp_units, return_sequences=False, dropout=hp_dropout))
    # Output
    model.add(tf.keras.layers.Dense(50, activation=hp_activation, kernel_regularizer=regularizers.L2(hp_kernel_reg)))  
    model.add(tf.keras.layers.Reshape((5, 10))) 
    model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                    optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                    metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

train_X_list=[]
train_Y_list=[]
val_X_list=[]
val_Y_list=[]
test_X_list=[]
test_Y_list=[] 
hp_lags=[1,3,6,12,18,24] 
for lag in hp_lags:
    train_X, train_Y, val_X, val_Y, test_X, test_Y = WindowData(lag)
    train_X_list.append(train_X)
    train_Y_list.append(train_Y)
    val_X_list.append(val_X)
    val_Y_list.append(val_Y)
    test_X_list.append(test_X)
    test_Y_list.append(test_Y)
    
    
best_hps_per_window = []
best_metrics_per_window = []
best_tuner_per_window = []
best_model_per_window = []

start_time = time.time()    
for i in range(len(hp_lags)):
    nombre_archivo = 'numero.csv'
    # csv stating which lag is currently working on
    with open(nombre_archivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Lag'])  
        writer.writerow([hp_lags[i]])    

    tuner = kt.GridSearch(model_builder,
                    objective='val_loss',
                   project_name=f'Search_{hp_lags[i]}_lags')
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    
    tuner.search(train_X_list[i], train_Y_list[i], epochs=200, validation_data=(val_X_list[i], val_Y_list[i]), callbacks=[stop_early])
    
    best_model=tuner.get_best_models(num_models=1)[0]
    best_model_per_window.append(best_model)
    
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    best_hps_per_window.append(best_hps)
    
    train_loss, train_mse = best_model.evaluate(train_X_list[i], train_Y_list[i])
    val_loss, val_mse = best_model.evaluate(val_X_list[i], val_Y_list[i])

    best_metrics_per_window.append({
        'train_loss': train_loss,
        'train_mse': train_mse,
        'val_loss': val_loss,
        'val_mse': val_mse
    })
    best_tuner_per_window.append(tuner)

best_window_index = min(range(len(best_metrics_per_window)), key=lambda i: best_metrics_per_window[i]['val_loss'])
best_hps = best_hps_per_window[best_window_index]
best_metrics = best_metrics_per_window[best_window_index]
best_model = best_model_per_window[best_window_index]
best_tuner = best_tuner_per_window[best_window_index]

print(f"Best window lag: {hp_lags[best_window_index]}")
print(f"Best hyperparameters: {best_hps.get('units')} units, {best_hps.get('activation')} activation, {best_hps.get('learning_rate')} learning rate, {best_hps.get('dropout')} dropout, {best_hps.get('kernel_regularizer')} regularizer.")
print(f"Best metrics: {best_metrics}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time elapsed: {elapsed_time}")

## Check history for overfitting

In [ ]:
# Seed is established so same results as best_model in tuning
train_X, train_Y, val_X, val_Y, test_X, test_Y = WindowData(hp_lags[best_window_index])
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model = model_builder(best_hps)
history = model.fit(train_X, train_Y, epochs=200, validation_data=(val_X, val_Y), callbacks=[stop_early])

val_mae_per_epoch = history.history['val_loss']
best_epoch = val_mae_per_epoch.index(min(val_mae_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
plt.plot(history.history['val_loss'], label="Validation")
plt.plot(history.history['loss'],label="Train")
plt.legend()
plt.axvline(best_epoch-1, ls="--",color="green")
plt.xlabel("Epochs")
plt.ylabel("MAE")

In [ ]:
eval_result = best_model.evaluate(test_X, test_Y)
print("[test loss, test MSE]:", eval_result)

In [ ]:
preds=best_model.predict(test_X)[0,:,:]
for i in range(preds.shape[1]):
    preds[:,i]=preds[:,i]*train_std[Housing][i]+train_mean[Housing][i]

In [ ]:
mean_absolute_error(preds,np.array(test[Housing]))

In [ ]:
mean_squared_error(preds,np.array(test[Housing]))

# Model trained with train + val

In [ ]:
train=df.iloc[:(n-5),]
train_mean = train.mean(); 
train_std = train.std();

train_df = (train - train_mean) / train_std
test_df = (test - train_mean) / train_std

train_X, train_Y, test_X, test_Y = WindowData(hp_lags[best_window_index],train_df=train_df,test_df=test_df,val=False)
model = model_builder(best_hps)
model.fit(x=train_X, y=train_Y, epochs=best_epoch)

preds_LSTM_1=model.predict(test_X)[0,:,:]
for i in range(preds_LSTM_1.shape[1]):
    preds_LSTM_1[:,i]=preds_LSTM_1[:,i]*train_std[Housing][i]+train_mean[Housing][i]

# Different initializations

MAE_test_1 = []
MSE_test_1 = []
for j in range(10):
    model = model_builder(best_hps, seedp=j)
    model.fit(x=train_X, y=train_Y, epochs=best_epoch)
    preds=model.predict(test_X)[0,:,:]
    for i in range(preds.shape[1]):
        preds[:,i]=preds[:,i]*train_std[Housing][i]+train_mean[Housing][i]
    MAE_test_1.append(mean_absolute_error(preds,np.array(test[Housing])))
    MSE_test_1.append(mean_squared_error(preds,np.array(test[Housing])))

# LSTM with 2 Layers

In [ ]:
def model_builder2(hp,seedp=0):
    # Seeds
    keras.utils.set_random_seed(100510766+seedp)
    # Model
    model = tf.keras.models.Sequential()
    # Hyperparameters
    units_choices = ['64_32','128_64','256_128','512_256','1024_512','1024_1024','2048_1024']
    selected_units = hp.Choice('units', units_choices)
    units = list(map(int, selected_units.split('_')))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 
    hp_activation = hp.Choice('activation', values=['relu','tanh']) 
    hp_dropout=hp.Choice('dropout',values=[float(0), 0.1, 0.2, 0.3]) 
    hp_kernel_reg = hp.Choice('kernel_regularizer', values=[float(0), 1e-3, 1e-2, 1e-1])
    # Layers
    model.add(tf.keras.layers.LSTM(units=units[0], return_sequences=True, dropout=hp_dropout))
    model.add(tf.keras.layers.LSTM(units=units[1], return_sequences=False, dropout=hp_dropout))
    # Output
    model.add(tf.keras.layers.Dense(50, activation=hp_activation, kernel_regularizer=regularizers.L2(hp_kernel_reg)))   
    model.add(tf.keras.layers.Reshape((5, 10))) 
    model.compile(loss=tf.keras.losses.MeanAbsoluteError(),
                    optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                    metrics=[tf.keras.metrics.MeanSquaredError()])
    return model

train_X_list=[]
train_Y_list=[]
val_X_list=[]
val_Y_list=[]
test_X_list=[]
test_Y_list=[] 
hp_lags=[1,3,6,12,18,24]
for lag in hp_lags:
    train_X, train_Y, val_X, val_Y, test_X, test_Y = WindowData(lag)
    train_X_list.append(train_X)
    train_Y_list.append(train_Y)
    val_X_list.append(val_X)
    val_Y_list.append(val_Y)
    test_X_list.append(test_X)
    test_Y_list.append(test_Y)
    
    
best_hps_per_window = []
best_metrics_per_window = []
best_tuner_per_window = []
best_model_per_window = []

start_time = time.time()    
for i in range(len(hp_lags)):
    nombre_archivo = 'numero.csv'
    # csv stating which lag is currently working on
    with open(nombre_archivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Lag'])  
        writer.writerow([hp_lags[i]])   

    tuner = kt.GridSearch(model_builder2,
                    objective='val_loss',
                   project_name=f'Search2_{hp_lags[i]}_lags')
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
    
    tuner.search(train_X_list[i], train_Y_list[i], epochs=200, validation_data=(val_X_list[i], val_Y_list[i]), callbacks=[stop_early])
    
    best_model=tuner.get_best_models(num_models=1)[0]
    best_model_per_window.append(best_model)
    
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    best_hps_per_window.append(best_hps)
    
    train_loss, train_mse = best_model.evaluate(train_X_list[i], train_Y_list[i])
    val_loss, val_mse = best_model.evaluate(val_X_list[i], val_Y_list[i])

    best_metrics_per_window.append({
        'train_loss': train_loss,
        'train_mse': train_mse,
        'val_loss': val_loss,
        'val_mse': val_mse
    })
    best_tuner_per_window.append(tuner)

best_window_index = min(range(len(best_metrics_per_window)), key=lambda i: best_metrics_per_window[i]['val_loss'])
best_hps = best_hps_per_window[best_window_index]
best_metrics = best_metrics_per_window[best_window_index]
best_model = best_model_per_window[best_window_index]
best_tuner = best_tuner_per_window[best_window_index]

print(f"Best window lag: {hp_lags[best_window_index]}")
print(f"Best hyperparameters: {best_hps.get('units')} units, {best_hps.get('activation')} activation, {best_hps.get('learning_rate')} learning rate, {best_hps.get('dropout')} dropout, {best_hps.get('kernel_regularizer')} regularizer.")
print(f"Best metrics: {best_metrics}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time elapsed: {elapsed_time}")

## Check history for overfitting

In [ ]:
# Seed is established so same results as best_model in tuning
train_X, train_Y, val_X, val_Y, test_X, test_Y = WindowData(hp_lags[best_window_index])
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model = model_builder2(best_hps)
history = model.fit(train_X, train_Y, epochs=200, validation_data=(val_X, val_Y), callbacks=[stop_early])

val_mae_per_epoch = history.history['val_loss']
best_epoch = val_mae_per_epoch.index(min(val_mae_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
plt.plot(history.history['val_loss'], label="Validation")
plt.plot(history.history['loss'],label="Train")
plt.legend()
plt.axvline(best_epoch-1, ls="--",color="green")
plt.xlabel("Epochs")
plt.ylabel("MAE")

In [ ]:
eval_result = best_model.evaluate(test_X, test_Y)
print("[test loss, test MSE]:", eval_result)

In [ ]:
preds_LSTM_2=best_model.predict(test_X)[0,:,:]
for i in range(preds_LSTM_2.shape[1]):
    preds_LSTM_2[:,i]=preds_LSTM_2[:,i]*train_std[Housing][i]+train_mean[Housing][i]

In [ ]:
mean_absolute_error(preds_LSTM_2,np.array(test[Housing]))

In [ ]:
mean_squared_error(preds_LSTM_2,np.array(test[Housing]))

# Model trained with train + val

In [ ]:
train=df.iloc[:(n-5),]
train_mean = train.mean(); 
train_std = train.std();

train_df = (train - train_mean) / train_std
test_df = (test - train_mean) / train_std

train_X, train_Y, test_X, test_Y = WindowData(hp_lags[best_window_index],train_df=train_df,test_df=test_df,val=False)
model = model_builder2(best_hps)
model.fit(x=train_X, y=train_Y, epochs=best_epoch)

preds_LSTM_1=model.predict(test_X)[0,:,:]
for i in range(preds_LSTM_1.shape[1]):
    preds_LSTM_1[:,i]=preds_LSTM_1[:,i]*train_std[Housing][i]+train_mean[Housing][i]

# Different initializations

MAE_test_2 = []
MSE_test_2 = []
for j in range(10):
    model = model_builder2(best_hps, seedp=j)
    model.fit(x=train_X, y=train_Y, epochs=best_epoch)
    preds=model.predict(test_X)[0,:,:]
    for i in range(preds.shape[1]):
        preds[:,i]=preds[:,i]*train_std[Housing][i]+train_mean[Housing][i]
    MAE_test_2.append(mean_absolute_error(preds,np.array(test[Housing])))
    MSE_test_2.append(mean_squared_error(preds,np.array(test[Housing])))

# Plotting all predictions

## Loading csv's with other models predicitons

In [ ]:
preds_baseline=pd.DataFrame([df[Housing][700:705].mean()]*5)
preds_DFM_forecast= pd.read_csv('C:/Users/Rolando/Desktop/y_DFM.csv')
preds_DFMN_SLBDD= pd.read_csv('C:/Users/Rolando/Desktop/y_DFMN.csv')
preds_GDFM= pd.read_csv('C:/Users/Rolando/Desktop/y_GDFM_factor.csv')
preds_GDFM_forecast= pd.read_csv('C:/Users/Rolando/Desktop/y_GDFM.csv')
preds_GDFM_SLBDD= pd.read_csv('C:/Users/Rolando/Desktop/y_GDFMN.csv')
preds_LSTM_1=pd.DataFrame(preds_LSTM_1)
preds_LSTM_2=pd.DataFrame(preds_LSTM_2)

In [ ]:
np.mean(MAE_test_2)

In [ ]:
np.std(MAE_test_2)

## Plots

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(15, 25))
for i in range(10):
    axes[i%5, int(i>4)].plot(preds_baseline.iloc[:,i], color="deeppink")
    axes[i%5, int(i>4)].plot(preds_DFM_forecast.iloc[:,i], color="orange")
    axes[i%5, int(i>4)].plot(preds_DFMN_SLBDD.iloc[:,i], color="sienna")
    axes[i%5, int(i>4)].plot(preds_GDFM.iloc[:,i], color="yellow")
    axes[i%5, int(i>4)].plot(preds_GDFM_SLBDD.iloc[:,i], color="purple")
    axes[i%5, int(i>4)].plot(preds_GDFM_forecast.iloc[:,i], color="lime")
    axes[i%5, int(i>4)].plot(preds_LSTM_1.iloc[:,i], color="red")
    axes[i%5, int(i>4)].plot(preds_LSTM_2.iloc[:,i], color="cyan")
    axes[i%5, int(i>4)].plot(np.array(test[Housing].iloc[:,i]), color="blue")
    axes[i%5, int(i>4)].set_title(f'{Housing[i]}')
fig.subplots_adjust(wspace=0.2)
axes.flatten()[-1].legend(["Baseline", "DFM_forecast", "DFM_SLBDD", "GDFM", "GDFM_forecast", 
                           "GDFM_SLBDD", "LSTM_1", "LSTM_2", "True value"], loc='upper left', bbox_to_anchor=(-1.3, -0.2), shadow=True, ncol=9)
plt.show()
